In [1]:
import ctypes
import os
import subprocess

import pygccxml
from pygccxml import declarations, parser, utils

import os
import subprocess

__here__ = os.path.abspath(".")# Configure the xml generatortry:    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()    gcc_path = os.path.abspath(gcc_path)    assert os.path.isfile(gcc_path)except subprocess.CalledProcessError:    passos.chdir(r"castxml\bin")generator_path, generator_name = utils.find_xml_generator()os.chdir(__here__)xml_generator_config = parser.xml_generator_configuration_t(    xml_generator_path=generator_path,    xml_generator=generator_name,    compiler_path=gcc_path,)

# Configure the xml generator
try:
    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()
    gcc_path = os.path.abspath(gcc_path)
    assert os.path.isfile(gcc_path)
except subprocess.CalledProcessError:
    pass

os.chdir(r"castxml\bin")
generator_path, generator_name = utils.find_xml_generator()
os.chdir(__here__)

xml_generator_config = parser.xml_generator_configuration_t(
    xml_generator_path=generator_path,
    xml_generator=generator_name,
    compiler_path=gcc_path
)

In [2]:
__here__ = os.path.abspath(".")# Configure the xml generatortry:    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()    gcc_path = os.path.abspath(gcc_path)    assert os.path.isfile(gcc_path)except subprocess.CalledProcessError:    passos.chdir(r"castxml\bin")generator_path, generator_name = utils.find_xml_generator()os.chdir(__here__)xml_generator_config = parser.xml_generator_configuration_t(    xml_generator_path=generator_path,    xml_generator=generator_name,    compiler_path=gcc_path,)

header = os.path.abspath("Development/CANapAPI.h")
os.path.exists(header)

with open(header, "r") as fid:
    data = fid.read()

In [3]:
base, ext = os.path.splitext(header)
header2 = os.path.abspath(f"{base}2{ext}")

In [4]:
# These are to make 
header_fix = """
// Push these out of the way
#define ASAP3_EXPORT
#define CALL_CONV
#define WINAPI

// Windows Path
#define _MAX_PATH 256
#define MAX_PATH 256
#define CALLBACK __stdcall

// Some typedefs
typedef int BOOL;
typedef unsigned long DWORD;
typedef unsigned char BYTE;
typedef unsigned int BOOLEAN;
typedef unsigned int UINT;
typedef unsigned long ULONG_PTR;
typedef ULONG_PTR DWORD_PTR;
#define NULL 0
"""

with open(header2, "w") as fid:
    fid.write(header_fix)
    fid.write(data)
header_declarations = parser.parse(files=[header2], config=xml_generator_config)
ns = header_declarations[0]

INFO Parsing source file "C:\Projects\python_CANape\Development\CANapAPI2.h" ... 


In [5]:
class ctypes_typedef_gen(object):
    def __init__(self, typedef):
        self.typedef = typedef

    @property
    def decl_str(self):
        return self.typedef.decl_type.decl_string

    @property
    def unsigned(self):
        return "unsigned" in self.decl_str

    @property
    def name(self):
        return self.typedef.name

    @property\s+
    def short_name(self):
        return self.name.split("_")[0]

    @property
    def c_type(self):
        if self.decl_str.startswith("::"):
            return self.decl_str.strip(":")
        if hasattr(ctypes, f"c_{self.short_name}"):
            return f"c_{self.short_name}"
        if hasattr(ctypes, f"c_{self.decl_str}"):
            return f"c_{self.decl_str}"
        if self.decl_str.endswith(" int"):
            if "unsigned" in n.decl_str:
                unsigned="u"
            else:
                unsigned=""
            base = n.decl_str.split()[0]
            c_type = f"c_{unsigned}{base}"
            if hasattr(ctypes, c_type):
                return c_type
            base = n.decl_str.split()[1]
            c_type = f"c_{unsigned}{base}"
            if hasattr(ctypes, c_type):
                return c_type
        if self.decl_str.endswith("*"):
            return "POINTER()"
        
        type_map = {
            "unsigned char": "c_ubyte",
            "void *": "c_void_p",
        }
        if self.decl_str in type_map:
            return type_map[self.decl_str]

        raise Exception(f"Unimplemented type {self.decl_str}")

    @property
    def python_line(self):
        return f"{self.name}=ctypes.{self.c_type}"

    def __repr__(self):
        return "<{}={}>".format(self.name, self.decl_str)


In [6]:
deffed = list()
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    n = ctypes_typedef_gen(typedef)
    if n.name.startswith("ON"):
        # Callbacks    if n.name == n.decl_str.strip("::"):
        #Structs
        continue
    if n.name == "FNCDIAGNOFIFICATION":
        #TODO
        continue
    print(f"{n.name}=ctypes.{n.c_type}")
    deffed.append(n.name)
    if n.name=="DBFileInfo":
        break

BOOL=ctypes.c_int
DWORD=ctypes.c_ulong
BYTE=ctypes.c_ubyte
BOOLEAN=ctypes.c_uint
UINT=ctypes.c_uint
ULONG_PTR=ctypes.c_ulong
DWORD_PTR=ctypes.ULONG_PTR
TApplicationType=ctypes.TApplicationType
ASAP3_EVENT_CODE=ctypes.ASAP3_EVENT_CODE
TApplicationID=ctypes.TApplicationID
TFormat=ctypes.TFormat
TValueType=ctypes.TValueType
TObjectType=ctypes.TObjectType
TAsap3DiagHdl=ctypes.c_ulong
EnRecorderState=ctypes.EnRecorderState
EnParamType=ctypes.EnParamType
DiagJobResponse=ctypes.DiagJobResponse
DiagNumericParamater=ctypes.NumericParamater
DiagNotificationStruct=ctypes.DiagNotificationStruct
TMeasurementListEntry=ctypes.MeasurementListEntry
MeasurementListEntries=ctypes.MeasurementListEntries
DBObjectInfo=ctypes.DBObjectInfo
DBFileInfo=ctypes.DBFileInfo


In [9]:
def cpptype_to_ctypes(t):
    if declarations.is_array(t):
        if isinstance(t.base, pygccxml.declarations.cpptypes.char_t):
            return "ctypes.c_char_p({})".format(t.size)
    if declarations.is_pointer(t):
        if hasattr(t, 'base'):
            if declarations.is_pointer(t.base):
                if hasattr(hasattr(t, 'base'), 'base') and t.base.base.decl_string  == "char":
                    return "ctypes.pointer(ctypes.c_char_p)"  
            if t.base.decl_string == "char":
                return "ctypes.c_char_p"
            if t.base.decl_string == "void": 
                return "ctypes.c_void_p"
        return "ctypes.POINTER({})".format(cpptype_to_ctypes(declarations.remove_pointer(t)))
    if declarations.is_const(t):
        return cpptype_to_ctypes(declarations.remove_const(t))
    if not hasattr(t, "decl_string"):
        return "!!!!"
    if t.decl_string.startswith("::"):
        return t.decl_string[2:]
    tp = t.decl_string.split(" ")
    if len(tp)==1:
        return "ctypes.c_" + tp[0]
        if tp[0] == "bool":
            return "ctypes.c_bool"
        if tp[0] == "char":
            return "ctypes.c_char"
        if tp[0] == "double":
            return "ctypes.c_double"
        if tp[0] == "int":
            return "ctypes.c_int"
    if len(tp)==2:
        if tp[1] == "volatile":
            return "ctypes.c_{}".format(tp[0])
        if tp[0] == "unsigned":
            if tp[1] == "char":
                tp[1] = "byte"
            return "ctypes.c_u{}".format(tp[1])
        if tp[1] == "int":
            return "ctypes.c_{}".format(tp[0])
    if len(tp)==3:
        if tp[1]=="unsigned":
            return "ctypes.c_u{}".format(tp[0])
    raise(Exception(t.decl_string))

In [19]:
i=1
for x in ns.classes():
    if declarations.is_struct(x): #x.name in ["Appversion", "TApplicationType", "TApplicationID", "TAsap3Hdl", "version_t"]:
        if len(x.name)==0:
            continue
        print("class {}(ctypes.Structure):".format(x.name))
        print("{}_fields_ = [".format(" "*4*i))
        for var in x.variables():
            print("{}(\"{}\", {}),".format(" "*(4*(i+1)),var.name,cpptype_to_ctypes(var.decl_type)))
        print("{}]".format(" "*(4*i)))
        print()

class TApplicationID(ctypes.Structure):
    _fields_ = [
        ("tApplicationType", TApplicationType),
        ("tApplicationPath", ctypes.c_char_p(256)),
    ]

class DiagJobResponse(ctypes.Structure):
    _fields_ = [
        ("job_responsestring", ctypes.c_char_p),
        ("job_responseValue", ctypes.c_double),
    ]

class NumericParamater(ctypes.Structure):
    _fields_ = [
        ("DiagNumeric", EnParamType),
        ("IVal", ctypes.c_int),
        ("UIVal", ctypes.c_uint),
        ("FVal", ctypes.c_float),
        ("DVal", ctypes.c_double),
        ("Values", NumericParamater::PValues),
    ]

class DiagNotificationStruct(ctypes.Structure):
    _fields_ = [
        ("DiagHandle", TAsap3DiagHdl),
        ("DiagState", eServiceStates),
        ("PrivateData", ctypes.c_void_p),
    ]

class MeasurementListEntry(ctypes.Structure):
    _fields_ = [
        ("taskId", ctypes.c_ushort),
        ("rate", ctypes.c_ulong),
        ("SaveFlag", BOOL),
        ("Disabled", BOOL),
      

In [35]:

tmp=list()
for x in ns.declarations:
    if declarations.is_enum(x): #x.name in ["Appversion", "TApplicationType", "TApplicationID", "TAsap3Hdl", "version_t"]:
        if hasattr(x, "get_name2value_dict"):
            print(x.get_name2value_dict())
            break
        if len(x.name)==0:
            continue
        if (x.name) == "TApplicationType":
            pass

{'eUNDEFINED': 0, 'eCANAPE': 1, 'eAPPLOCATION': 3}


In [47]:
print(f"""{x.name}=generate_fancy_enum_factory(
    {{""")
for key, value in x.get_name2value_dict().items():
    print(f'       {value}: "{key}",')

print(f"""    }},
    "{x.name}",
)""")
                                              

TApplicationType=generate_fancy_enum_factory(
    {
       0: "eUNDEFINED",
       1: "eCANAPE",
       3: "eAPPLOCATION",
    },
    "TApplicationType",
)


In [50]:
def generate_fancy_enum_factory_generator(x):
    print(f"""
{x.name}=generate_fancy_enum_factory(
    {{""")
    for key, value in x.get_name2value_dict().items():
        print(f'       {value}: "{key}",')

    print(f"""    }},
    "{x.name}",
)""")
          

In [51]:
generate_fancy_enum_factory_generator(x)


TApplicationType=generate_fancy_enum_factory(
    {
       0: "eUNDEFINED",
       1: "eCANAPE",
       3: "eAPPLOCATION",
    },
    "TApplicationType",
)


In [53]:

tmp=list()
for x in ns.declarations:
    if declarations.is_enum(x): #x.name in ["Appversion", "TApplicationType", "TApplicationID", "TAsap3Hdl", "version_t"]:
        if hasattr(x, "get_name2value_dict"):
            generate_fancy_enum_factory_generator(x)


TApplicationType=generate_fancy_enum_factory(
    {
       0: "eUNDEFINED",
       1: "eCANAPE",
       3: "eAPPLOCATION",
    },
    "TApplicationType",
)

TScriptStatus=generate_fancy_enum_factory(
    {
       1: "eTScrReady",
       2: "eTScrStarting",
       3: "eTScrRunning",
       4: "eTScrSleeping",
       5: "eTScrSuspended",
       6: "eTScrTerminated",
       7: "eTScrFinishedReturn",
       8: "eTScrFinishedCancel",
       9: "eTScrFailure",
       10: "eTScrTimeout",
    },
    "TScriptStatus",
)

e_RamMode=generate_fancy_enum_factory(
    {
       0: "e_TR_MODE_RAM",
       1: "e_TR_MODE_ROM",
    },
    "e_RamMode",
)

TRecorderType=generate_fancy_enum_factory(
    {
       0: "eTRecorderTypeMDF",
       1: "eTRecorderTypeILinkRT",
       2: "eTRecorderTypeBLF",
    },
    "TRecorderType",
)

ASAP3_EVENT_CODE=generate_fancy_enum_factory(
    {
       0: "et_ON_DATA_ACQ_START",
       1: "et_ON_DATA_ACQ_STOP",
       2: "et_ON_BEFORE_DATA_ACQ_START",
       3: "et_ON_CL

In [31]:
                                              

        0: "UNDEFINED",
        1: "CANAPE",
        3: "APPLOCATION",
    },
    "ApplicationType"    
))

append_value: <class 'method'><<bound method enumeration_t.append_value of <pygccxml.declarations.enumeration.enumeration_t object at 0x00000184D38E5B00>>>
attributes: <class 'NoneType'><None>
byte_align: <class 'float'><4.0>
byte_size: <class 'float'><4.0>
cache: <class 'pygccxml.declarations.algorithms_cache.declaration_algs_cache_t'><<pygccxml.declarations.algorithms_cache.declaration_algs_cache_t object at 0x00000184D38E5AC8>>
create_decl_string(): ::TApplicationType
decl_string: <class 'str'><::TApplicationType>
decorated_name: <class 'NoneType'><None>
demangled: <class 'NoneType'><None>
elaborated_type_specifier: <class 'str'><enum>
get_mangled_name(): None
get_name2value_dict(): {'eUNDEFINED': 0, 'eCANAPE': 1, 'eAPPLOCATION': 3}
has_value_name: <class 'method'><<bound method enumeration_t.has_value_name of <pygccxml.declarations.enumeration.enumeration_t object at 0x00000184D38E5B00>>>
i_depend_on_them(): []
is_artificial: <class 'bool'><False>
location: <class 'pygccxml.declara

C:\Projects\bootstrap_WinPython\WinPython\python-3.6.6.amd64\lib\site-packages\pygccxml\declarations\declaration.py:303: DeprecationWarning: The decorated_name attribute is deprecated. See the changelog.
  DeprecationWarning)


In [14]:
for var in ns.typedefs():
    if var.name.startswith("_"):
        continue
    if var.name.startswith("ON"):
        # Callbacks
        continue
    if var.name == var.decl_type.decl_string.strip("::"):
        # Structs
        continue
    if var.name == "FNCDIAGNOFIFICATION":
        #todo
        continue
    c_type = (cpptype_to_ctypes(var.decl_type))
    print(f"{var.name}={c_type}")

BOOL=ctypes.c_int
DWORD=ctypes.c_ulong
BYTE=ctypes.c_ubyte
BOOLEAN=ctypes.c_uint
UINT=ctypes.c_uint
ULONG_PTR=ctypes.c_ulong
DWORD_PTR=ULONG_PTR
TAsap3DiagHdl=ctypes.c_ulong
DiagNumericParamater=NumericParamater
TMeasurementListEntry=MeasurementListEntry
TAsap3Hdl=ctypes.POINTER(tAsap3Hdl)
TRecorderID=ctypes.POINTER(ctypes.c_ulong)
TModulHdl=ctypes.c_ushort
TScriptHdl=DWORD_PTR
TTime=ctypes.c_ulong


In [15]:
for x in ns.declarations:
    if declarations.is_enum(x):
        print(x)
        break

TApplicationType [enumeration]


In [16]:
x.decl_string

'::TApplicationType'

In [17]:
x.partial_decl_string

'::TApplicationType'

In [18]:
x.elaborated_type_specifier

'enum'

In [59]:
print(x)

TLayoutCoeffs [struct]


In [57]:
declarations.is_enum(x)

False

In [59]:
tmp=list()
for x in ns.declarations:
    if declarations.is_enum(x): #x.name in ["Appversion", "TApplicationType", "TApplicationID", "TAsap3Hdl", "version_t"]:
        print(f"|{x.name}|")
    if x.name.strip()=="TApplicationType":
        tmp.append(x)
tmp

|TApplicationType|
|TApplicationType|
|TScriptStatus|
|e_RamMode|
|TRecorderType|
|ASAP3_EVENT_CODE|
|ASAP3_EVENT_CODE|
|TFormat|
|TFormat|
|TValueType|
|TValueType|
|TObjectType|
|TObjectType|
|eServiceStates|
|EnRecorderState|
|EnRecorderState|
|EnParamType|
|EnParamType|
|TAsap3FileType|
|TAsap3FileType|
|TAsap3ECUState|
|TAsap3ECUState|
|TAsap3DataType|
|TAsap3DataType|
|TAsap3DBOType|
|TAsap3DBOType|
|tDriverType|
|tDriverType|
|tMeasurementState|
|tMeasurementState|


In [60]:
x = tmp[1]

In [64]:
x.i_depend_on_them()[0]

In [73]:
x.location.as_tuple()

('C:\\Projects\\python_CANape\\Development\\CANapAPI2.h', 267)

In [70]:
vardump(x.i_depend_on_them()[0].declaration)

attributes: <class 'NoneType'><None>
byte_align: <class 'float'><4.0>
byte_size: <class 'float'><4.0>
cache: <class 'pygccxml.declarations.algorithms_cache.declaration_algs_cache_t'><<pygccxml.declarations.algorithms_cache.declaration_algs_cache_t object at 0x00000184D38E5CC0>>
create_decl_string(): ::TApplicationType
decl_string: <class 'str'><::TApplicationType>
decl_type: <class 'pygccxml.declarations.cpptypes.declarated_t'><TApplicationType>
decorated_name: <class 'NoneType'><None>
demangled: <class 'NoneType'><None>
get_mangled_name(): None
i_depend_on_them(): [<pygccxml.declarations.class_declaration.dependency_info_t object at 0x00000184D3958C88>]
is_artificial: <class 'bool'><False>
location: <class 'pygccxml.declarations.location.location_t'><<pygccxml.declarations.location.location_t object at 0x00000184D38E5C88>>
mangled: <class 'NoneType'><None>
name: <class 'str'><TApplicationType>
parent: <class 'pygccxml.declarations.namespace.namespace_t'><:: [namespace]>
partial_decl_s

C:\Projects\bootstrap_WinPython\WinPython\python-3.6.6.amd64\lib\site-packages\pygccxml\declarations\declaration.py:303: DeprecationWarning: The decorated_name attribute is deprecated. See the changelog.
  DeprecationWarning)


In [62]:
vardump(x).i_depend_on_them()

attributes: <class 'NoneType'><None>
byte_align: <class 'float'><4.0>
byte_size: <class 'float'><4.0>
cache: <class 'pygccxml.declarations.algorithms_cache.declaration_algs_cache_t'><<pygccxml.declarations.algorithms_cache.declaration_algs_cache_t object at 0x00000184D38E5CC0>>
create_decl_string(): ::TApplicationType
decl_string: <class 'str'><::TApplicationType>
decl_type: <class 'pygccxml.declarations.cpptypes.declarated_t'><TApplicationType>
decorated_name: <class 'NoneType'><None>
demangled: <class 'NoneType'><None>
get_mangled_name(): None
i_depend_on_them(): [<pygccxml.declarations.class_declaration.dependency_info_t object at 0x00000184D39651D0>]
is_artificial: <class 'bool'><False>
location: <class 'pygccxml.declarations.location.location_t'><<pygccxml.declarations.location.location_t object at 0x00000184D38E5C88>>
mangled: <class 'NoneType'><None>
name: <class 'str'><TApplicationType>
parent: <class 'pygccxml.declarations.namespace.namespace_t'><:: [namespace]>
partial_decl_s

C:\Projects\bootstrap_WinPython\WinPython\python-3.6.6.amd64\lib\site-packages\pygccxml\declarations\declaration.py:303: DeprecationWarning: The decorated_name attribute is deprecated. See the changelog.
  DeprecationWarning)


AttributeError: 'NoneType' object has no attribute 'i_depend_on_them'